In [ ]:
# !pip install requests

### Importing Required Libraries

In [1]:
import requests

#### Defining the required API & Endpoint keys

In [2]:
AZURE_VISION_ENDPOINT = "https://mindsartapi-imagecaption-2.cognitiveservices.azure.com/vision/v3.1/analyze?visualFeatures=Description"
AZURE_VISION_API_KEY = "AdQAfeZyRq0km3uH1KMxdBXAgC76e4i0VulxxmntLbw55fZdPL82JQQJ99BCACfhMk5XJ3w3AAAFACOGkC8v"

AZURE_OPENAI_GPT4_ENDPOINT = "https://mindsart-storygeneration.openai.azure.com/openai/deployments/mindsArt-StoryGeneration-GPT4/chat/completions?api-version=2024-08-01-preview"
AZURE_OPENAI_GPT4_API_KEY = "lGsXrBI6BLLSaVNeTk6ILqt24GSpT0koiqRP7iINuB5aqKqzf5XDJQQJ99BCACfhMk5XJ3w3AAABACOGWSmU"

### Extract Caption from a Local Image (Azure Vision API)

In [3]:
def extract_caption_from_image(ApiKey, EndpointKey,image_path):
    headers = {
        "Ocp-Apim-Subscription-Key": ApiKey,
        "Content-Type": "application/octet-stream"
    }

    with open(image_path, "rb") as image_file:
        image_data = image_file.read()

    response = requests.post(EndpointKey, headers=headers, data=image_data)
    response_data = response.json()

    if "description" in response_data and response_data["description"]["captions"]:
        return response_data["description"]["captions"][0]["text"]
    return "No caption found."

In [4]:
# Example usage
image_path = "img_0.png" # Image path will be different
caption = extract_caption_from_image(ApiKey=AZURE_VISION_API_KEY, EndpointKey=AZURE_VISION_ENDPOINT, image_path=image_path)
print("Extracted Caption:", caption)

Extracted Caption: a painting of a man in a robe and a hat


### Caption Enhance Using GPT Model

In [5]:
def enhance_caption(ApiKey, EndpointKey, image_caption, original_prompt):
    headers = {
        "Content-Type": "application/json",
        "api-key": ApiKey
    }

    data = {
        "messages": [
            {"role": "system", "content": "You are a helpful AI that improves image captions."},
            {"role": "user", "content": f"Improve this image caption using the context: \n\nOriginal Caption: {image_caption}\nOriginal Prompt: {original_prompt}\n\nMake it more detailed and creative."}
        ],
        "max_tokens": 100
    }

    response = requests.post(EndpointKey, headers=headers, json=data)

    if response.status_code == 200:
        result = response.json()
        return result["choices"][0]["message"]["content"]
    else:
        return f"Error: {response.text}"

In [6]:
#Example Usage
#Original prompt is generated by the user in create image phase
original_prompt = "painting of King Henry VIII carrying an umbrella"
enhance_caption = enhance_caption(ApiKey=AZURE_OPENAI_GPT4_API_KEY,
                                  EndpointKey=AZURE_OPENAI_GPT4_ENDPOINT,
                                  image_caption=caption,
                                  original_prompt=original_prompt)
print("Enhanced Caption:", enhance_caption)

Enhanced Caption: Enhanced Caption: An exquisite oil painting capturing King Henry VIII's stoic pose, adorned in a richly textured robe and a distinctive hat, reassuringly clutching a grand umbrella.


### Story Generation Based on GPT model

In [7]:
def generate_story(ApiKey, EndpointKey,prompt: str, max_tokens=300, temperature=0.7, top_p=0.9):
    """
    Generate a creative story using Azure OpenAI's GPT model.

    Parameters:
    - prompt (str): The story prompt provided by the user.
    - max_tokens (int): Maximum length of the generated text.
    - temperature (float): Creativity level (higher = more creative).
    - top_p (float): Controls diversity of responses.

    Returns:
    - str: The generated story or an error message.
    """

    headers = {
        "Content-Type": "application/json",
        "api-key": ApiKey
    }

    data = {
        "messages": [
            {"role": "system", "content": "You are a creative story writer."},
            {"role": "user", "content": prompt}
        ],
        "max_tokens": max_tokens,
        "temperature": temperature,
        "top_p": top_p,
        "n": 1
    }

    response = requests.post(EndpointKey, headers=headers, json=data)

    if response.status_code == 200:
        result = response.json()
        return result["choices"][0]["message"]["content"]
    else:
        return f"Error: {response.text}"

In [8]:
# Example Usage
story_prompt = f"Write a creative story about {enhance_caption}."
story = generate_story(ApiKey=AZURE_OPENAI_GPT4_API_KEY,
                       EndpointKey=AZURE_OPENAI_GPT4_ENDPOINT,
                       prompt=story_prompt)
print(story)

Title: The Umbrella of Power

Once upon a time, in the kingdom of England, there lived a mighty and charismatic king named Henry VIII. He was known for his robust personality, his love for grandeur, and his distinctive hat that he wore as a symbol of his royal authority. However, there was something else that set him apart - an unusual artifact, a grand umbrella.

One rainy afternoon, the palace's chief artist, Sir William, was commissioned to create a portrait of King Henry VIII. The king wanted a painting that would not just capture his physical likeness, but also his spirit - stoic, yet warm; commanding, yet approachable. Sir William was a talented man, known for his ability to breathe life into his paintings. He accepted the challenge and began his work.

He painted the king in a richly textured robe, a deep shade of royal blue, with intricate gold embroidery. The king's hat, a distinctive piece made of the finest velvet, sat proudly on his head. But when it came to the grand umbre